# This notebook is mainly used for the capstone project

In [ ]:
import pandas as pd


In [ ]:
import numpy as np

In [ ]:
print("Hello Capstone Project Course!")

# All the 3 parts of the project are done in a single notebook.

# 1. Segmenting and Clustering Neighborhoods in Toronto

In [2]:
import pandas as pd
import numpy as np 
from pandas.io.json import json_normalize
import json
# Used https://beautiful-soup-4.readthedocs.io/en/latest/
#importing soup libraries
from bs4 import BeautifulSoup
import requests




# Extracting data from Wikipedia page

In [4]:
wiki_url= "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url_result= requests.get(wiki_url).text
soup = BeautifulSoup(url_result,"lxml")
#print(soup.prettify())
 

In [6]:
#Only process the cells that have an assigned borough
#Taken help from https://stackoverflow.com/questions/23377533/python-beautifulsoup-parsing-table
wiki_table= soup.find("table",class_="wikitable")
Nhood_rows=wiki_table.find_all("tr")

#Trying to extract the column names(Postcode,Borough,Neighbourhood)
Nhood=[]
for i in Nhood_rows:
    #deletes the first and last items with empty strings
    res=i.text.split("\n")[1:-1]
    Nhood.append(res)

Nhood[0:11]

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M8A', 'Not assigned', 'Not assigned']]

# Converting the data to Pandas dataframe.

In [7]:
Nhood[0][-1]="Neighborhood"
df_Nhood=pd.DataFrame(Nhood[1:],columns=Nhood[0])
#Prints Borough and Neighborhood 'Not assigned' values too
df_Nhood.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [8]:
#Removing 'Not assigned' from Borough and Neighborhood
borough_notassigned=df_Nhood.index[df_Nhood["Borough"]=="Not assigned"]
nhood__notassigned=df_Nhood.index[df_Nhood["Neighborhood"]=="Not assigned"]
BoroughNhood_notassigned=borough_notassigned & nhood__notassigned

#This only process the cells that have an assigned borough. We ignore cells with a borough that is Not assigned.
df_Nhood.drop(df_Nhood.index[borough_notassigned],inplace=True)
df_Nhood.reset_index(drop=True,inplace=True)
df_Nhood.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [10]:
nhood__notassigned=df_Nhood.index[df_Nhood["Neighborhood"]=="Not assigned"]
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
for j in nhood__notassigned:
    df_Nhood["Neighborhood"][j]=df_Nhood["Borough"][j]

df_Nhood.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [11]:
#Grouping the neighbourhoods with the same postal code. Separated by comma.
postalCode_group=df_Nhood.groupby("Postcode")
Nhood_group=postalCode_group['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
borough_group=postalCode_group['Borough'].apply(lambda x: set(x).pop())
df_group= pd.DataFrame(list(zip(borough_group.index,borough_group,Nhood_group)))
df_group.columns=["Postcode", "Borough", "Neighborhood"]
df_group.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [14]:
#using the .shape method to print the number of rows of your dataframe
df_group.shape

(103, 3)

# 2. The latitude and the longitude coordinates of each neighborhood.

In [28]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
postcode_coordinates=df_group.join(df_coordinates.set_index("Postal Code"),on="Postcode")
postcode_coordinates.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
